# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using surprise. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives
You will be able to:
* Fit a recommender system model to a set of data
* Create a function that will return the top recommendations for a user
* Introduce a new user to a rating matrix and make recommendations for them

For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last  lesson, you got exposed to working with Surprise datasets. In this lab, you will also go through the process of reading in a dataset into the Surprise dataset format. To begin with, load the dataset into a pandas dataframe. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
#drop unnecessary columns
new_df = df.drop('timestamp', axis=1)

It's now time to transform the dataset into something compatible with Surprise. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [3]:
from surprise import Reader, Dataset
reader = Reader(line_format='user item rating')
movies = Dataset.load_from_df(new_df, reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [4]:
dataset = movies.build_full_trainset()
print('Number of users: ',dataset.n_users,'\n')
print('Number of items: ',dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the Best Model
Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [5]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise import accuracy
import numpy as np

In [6]:
## Perform a gridsearch with SVD
param_grid = {'n_factors' :[20,50,100],
              'reg_all':[0.02,0.05,0.1]}
gs_model = GridSearchCV(SVD, param_grid=param_grid, n_jobs = -1, joblib_verbose=5)
gs_model.fit(movies)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  1.9min finished


In [7]:
# print out optimal parameters for SVD after GridSearch
print(gs_model.best_score)
print(gs_model.best_params)

{'rmse': 0.8690057211713708, 'mae': 0.6681603962654936}
{'rmse': {'n_factors': 20, 'reg_all': 0.05}, 'mae': {'n_factors': 20, 'reg_all': 0.05}}


In [8]:
# cross validating with KNNBasic
sim_pearson = {'name':'pearson','user_based':True}
knn_basic = KNNBasic(sim_options=sim_pearson)
cv_knn_basic= cross_validate(knn_basic, movies, n_jobs=-1)

In [9]:
cv_knn_basic

{'test_rmse': array([0.97236236, 0.97252461, 0.96294686, 0.98504599, 0.96956486]),
 'test_mae': array([0.74970323, 0.7503992 , 0.74430147, 0.76011312, 0.75100912]),
 'fit_time': (0.4925103187561035,
  0.5512731075286865,
  0.5437541007995605,
  0.5013909339904785,
  0.4603002071380615),
 'test_time': (1.485041856765747,
  1.5556869506835938,
  1.4402470588684082,
  1.3679001331329346,
  1.2613270282745361)}

In [10]:
cv_knn_basic.items()

dict_items([('test_rmse', array([0.97236236, 0.97252461, 0.96294686, 0.98504599, 0.96956486])), ('test_mae', array([0.74970323, 0.7503992 , 0.74430147, 0.76011312, 0.75100912])), ('fit_time', (0.4925103187561035, 0.5512731075286865, 0.5437541007995605, 0.5013909339904785, 0.4603002071380615)), ('test_time', (1.485041856765747, 1.5556869506835938, 1.4402470588684082, 1.3679001331329346, 1.2613270282745361))])

In [11]:
# print out the average RMSE score for the test set
for val in cv_knn_basic.items():
    print(val)
print('------------------------')
print(cv_knn_basic['test_rmse'].mean())

('test_rmse', array([0.97236236, 0.97252461, 0.96294686, 0.98504599, 0.96956486]))
('test_mae', array([0.74970323, 0.7503992 , 0.74430147, 0.76011312, 0.75100912]))
('fit_time', (0.4925103187561035, 0.5512731075286865, 0.5437541007995605, 0.5013909339904785, 0.4603002071380615))
('test_time', (1.485041856765747, 1.5556869506835938, 1.4402470588684082, 1.3679001331329346, 1.2613270282745361))
------------------------
0.9724889377135545


In [12]:
# cross validating with KNNBaseline
sim_pearson = {'name':'pearson','user_based':True}
knn_baseline = KNNBaseline(sim_options=sim_pearson)
cv_knn_baseline = cross_validate(knn_baseline, movies, n_jobs=-1)

In [13]:
# print out the average score for the test set
for val in cv_knn_baseline.items():
    print(val)
print('-------------------')
print(cv_knn_baseline['test_rmse'].mean())

('test_rmse', array([0.87771672, 0.87114056, 0.87578025, 0.87718614, 0.87801323]))
('test_mae', array([0.67031225, 0.66754468, 0.66912055, 0.66826808, 0.67124048]))
('fit_time', (0.5382504463195801, 0.8538599014282227, 0.7899258136749268, 0.720128059387207, 0.5284879207611084))
('test_time', (2.6848459243774414, 2.906275987625122, 3.215756893157959, 2.3935561180114746, 1.8500759601593018))
-------------------
0.8759673807836551


Based off these outputs, it seems like the best performing model is the SVD model with n_factors = 50 and a regularization rate of 0.05. Let's use that model to make some predictions. Use that model or if you found one that performs better, feel free to use that.

## Making Recommendations

This next section is going to involve making recommendations, and it's important that the output for the recommendation is interpretable to people. Rather than returning the movie_id values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [14]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [15]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [16]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [17]:
svd.predict(2,4)

Prediction(uid=2, iid=4, r_ui=None, est=3.085834897046554, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put all of our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step to go let's create a function that allows students to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater` should take as parameters:
* movie_df : DataFrame - a dataframe containing the movie ids, name of movie, and genres
* num : int - number of ratings
* genre : string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int  , 'movieId': int  ,'rating': float  }

#### This function is optional, but fun :) 

In [21]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list   

In [22]:
user_rating = movie_rater(movies, 4, 'Comedy')

TypeError: 'DatasetAutoFolds' object is not subscriptable

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [26]:
user_rating = [{'userId': 1000, 'movieId': 55245, 'rating': '5'},
                {'userId': 1000, 'movieId': 2491, 'rating': '4'},
                {'userId': 1000, 'movieId': 4718, 'rating': '4'},
                {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

In [27]:
user_rating

[{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a Surprise dataset
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number)

In [43]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating,ignore_index=True)
new_movies_ratings = Dataset.load_from_df(new_ratings_df, Reader())

In [44]:
# train a model using the new combined DataFrame
new_dataset = new_movies_ratings.build_full_trainset()
svd_ = SVD(n_factors=50, reg_all=0.05)
svd_.fit(new_dataset)

In [48]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_ratings_df['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))

In [56]:
# order the predictions from highest to lowest rated
ranked_movies = sorted(list_of_movies, key=lambda x: x[1], reverse=True)

 For the final component of this challenge, it could be useful to create a function `recommended_movies` that takes in the parameters:
* `user_ratings` : list - list of tuples formulated as (movie_id, rating) (should be in order of best to worst for this individual)
* `movie_title_df` : DataFrame 
* `n` : int- number of recommended movies 

The function should use a for loop to print out each recommended *n* movies in order from best to worst

In [60]:
# return the top n recommendations using the 
def recommended_movies(user_ratings, movie_title_df, n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ',idx+1,': ',title,'\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies, df_movies, 5)

Recommendation #  1 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  2 :  906    Lawrence of Arabia (1962)
Name: title, dtype: object 

Recommendation #  3 :  3622    Amelie (Fabuleux destin d'Amélie Poulain, Le) ...
Name: title, dtype: object 

Recommendation #  4 :  961    Great Escape, The (1963)
Name: title, dtype: object 

Recommendation #  5 :  841    Streetcar Named Desire, A (1951)
Name: title, dtype: object 



## Level Up

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top n recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the change to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will get exposed to using spark to make recommender systems.